## 导入包

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import copy 
import fnmatch
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [2]:
def reshape_rand(Rand, patterns):
    col_list = list(Rand.columns)
    id_columns = ['hhidpn']
    short_list = []
    for pat in patterns:
        short_list = short_list + fnmatch.filter(col_list, pat)

    Rand_small = Rand[id_columns + short_list]
    indexed_df = Rand_small.set_index('hhidpn')
    stacked_df = indexed_df.stack(dropna=False)
    long_df = stacked_df.reset_index()
    long_df['wave'] = long_df.loc[:,'level_1'].str.extract(r'(\d\d|\d)', expand=False)
    long_df['level_1'] = long_df['level_1'].str.replace('\d+', '', regex=True)
    features = list(long_df.level_1.unique())
    long_df = long_df.rename(columns = {0: 'value'})
    long_df_id = long_df[['hhidpn', 'wave']].drop_duplicates().reset_index(drop = True)
    
    for i in range(len(features)):
        temp_df = long_df.loc[long_df['level_1'] == features[i]].reset_index(drop = True)
        if i == 0:
            final_df = long_df_id.merge(temp_df, on = ['hhidpn', 'wave'], how = 'outer')
            final_df = final_df.rename(columns = {'value': features[i]})
        else:
            final_df = final_df.merge(temp_df, on = ['hhidpn', 'wave'], how = 'outer')
            final_df = final_df.rename(columns = {'value': features[i]})
            if 'level_1' in final_df.columns:
                final_df = final_df.drop('level_1', axis=1)
            
    return final_df[['hhidpn', 'wave'] + features]

## 读取数据

In [3]:
HRS_Rand = pd.read_stata('../data/randhrs1992_2020v1_STATA/randhrs1992_2020v1.dta')

In [5]:
HRS_Rand.shape

(42406, 17013)

In [44]:
hrs_patterns = ['r*wgihr', # 小时工资
                'r*unemp', # 是否无工作
                'r*lbrf', # 工作状态（全职/兼职）
                'r*cenreg', # 居住地区
                'r*mstat', # 婚姻状况
                'h*atotb', # 总财富
                'r*bmi', # BMI
                'r*shlt', # 自评健康状况
                'r*hltc3', # 自评健康状况变化
                'r*smokev', # 是否曾吸烟
                'r*higov', # 是否有政府健康保险
                'r*covr', # 是否有公司健康保险
               ]

In [45]:
long_hrs_rand = reshape_rand(HRS_Rand, hrs_patterns)
long_hrs_rand['hhidpn'] = long_hrs_rand['hhidpn'].apply(lambda x: str(x).zfill(9))
long_hrs_rand['wave'] = long_hrs_rand['wave'].astype(float)
long_hrs_rand.groupby('wave').count()

,hhidpn,rwgihr,runemp,rlbrf,rinlbrf,rcenreg,recenreg,rmstat,remstat,hatotb,rbmi,rpmbmi,rshlt,rhltc,rsmokev,rhigov,rehigov,rcovr,recovr
wave,,,,,,,,,,,,,,,,,,,
1.0,42406,7786,8783,12652,12637,12652,0,12652,0,12652,12652,0,12652,12651,12652,12445,0,12464,0
2.0,42406,7140,8147,16488,16443,19641,0,19642,0,19642,19434,0,19632,19631,19571,19587,0,11341,0
3.0,42406,6008,7048,17991,17638,17989,0,17928,0,17991,17773,0,17984,17785,17731,17912,0,16010,0
4.0,42406,7410,8632,21384,20919,21378,0,21360,0,21384,21099,0,21378,16284,21175,21274,0,20687,0
5.0,42406,6385,7407,19578,19092,19566,0,19558,0,19578,19289,0,19570,19299,19389,19491,0,19446,0
6.0,42406,5205,6304,18165,17683,18145,0,18150,0,18165,17818,0,18154,17920,17986,18118,0,17959,0
7.0,42406,6746,8152,20129,19653,20113,0,20114,0,20129,19750,0,20112,16642,19961,20052,0,19931,0
8.0,42406,5762,6858,18469,18019,18456,0,18467,0,18469,18175,7128,18444,18236,18313,18417,0,18301,0
9.0,42406,5216,6114,17217,16756,17205,0,17215,0,17217,16983,6446,17203,17053,17072,17154,0,17049,0


In [46]:
long_hrs_rand.to_csv('df_long.csv', index=False)

## 数据清洗

读取数据。

In [47]:
# df = long_hrs_rand[:].copy(deep=True)
df = pd.read_csv('df_long.csv')
df

,hhidpn,wave,rwgihr,runemp,rlbrf,rinlbrf,rcenreg,recenreg,rmstat,remstat,hatotb,rbmi,rpmbmi,rshlt,rhltc,rsmokev,rhigov,rehigov,rcovr,recovr
0,1010,1.0,15.0,0.no,1.works ft,1.yes,1.northeast,NaN,5.divorced,NaN,6000.000000,30.7,NaN,4.fair,2.about the same,1.yes,0.no,NaN,1.yes,NaN
1,1010,2.0,NaN,NaN,6.disabled,NaN,1.northeast,NaN,5.divorced,NaN,-69637.584585,24.4,NaN,4.fair,1.better,1.yes,0.no,NaN,1.yes,NaN
2,1010,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1010,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1010,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678491,959738010,12.0,NaN,NaN,5.retired,0.no,3.south,NaN,5.divorced,NaN,82000.000000,28.6,NaN,4.fair,3.worse,1.yes,1.yes,NaN,0.no,NaN
678492,959738010,13.0,NaN,NaN,5.retired,0.no,3.south,NaN,5.divorced,NaN,110000.000000,28.3,28.2,4.fair,2.about the same,1.yes,1.yes,NaN,0.no,NaN
678493,959738010,14.0,NaN,NaN,5.retired,0.no,3.south,NaN,5.divorced,NaN,167000.000000,24.0,NaN,4.fair,2.about the same,1.yes,1.yes,NaN,0.no,NaN
678494,959738010,15.0,NaN,NaN,5.retired,0.no,3.south,NaN,5.divorced,NaN,186500.000000,22.5,NaN,4.fair,3.worse,1.yes,1.yes,NaN,0.no,NaN


删除wave为缺失值的行。

In [48]:
df = df[~df.wave.isna()]
df

,hhidpn,wave,rwgihr,runemp,rlbrf,rinlbrf,rcenreg,recenreg,rmstat,remstat,hatotb,rbmi,rpmbmi,rshlt,rhltc,rsmokev,rhigov,rehigov,rcovr,recovr
0,1010,1.0,15.000000,0.no,1.works ft,1.yes,1.northeast,NaN,5.divorced,NaN,6000.000000,30.7,NaN,4.fair,2.about the same,1.yes,0.no,NaN,1.yes,NaN
1,1010,2.0,NaN,NaN,6.disabled,NaN,1.northeast,NaN,5.divorced,NaN,-69637.584585,24.4,NaN,4.fair,1.better,1.yes,0.no,NaN,1.yes,NaN
2,1010,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1010,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1010,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678490,959738010,11.0,7.853063,1.yes,3.unemployed,1.yes,3.south,NaN,5.divorced,NaN,77000.000000,28.3,NaN,4.fair,2.about the same,1.yes,1.yes,NaN,0.no,NaN
678491,959738010,12.0,NaN,NaN,5.retired,0.no,3.south,NaN,5.divorced,NaN,82000.000000,28.6,NaN,4.fair,3.worse,1.yes,1.yes,NaN,0.no,NaN
678492,959738010,13.0,NaN,NaN,5.retired,0.no,3.south,NaN,5.divorced,NaN,110000.000000,28.3,28.2,4.fair,2.about the same,1.yes,1.yes,NaN,0.no,NaN
678493,959738010,14.0,NaN,NaN,5.retired,0.no,3.south,NaN,5.divorced,NaN,167000.000000,24.0,NaN,4.fair,2.about the same,1.yes,1.yes,NaN,0.no,NaN


新建用于cox回归的数据框。

In [49]:
df_cleaned = pd.DataFrame(df.groupby('hhidpn')[['rwgihr']].count()).reset_index()
df_cleaned.columns = ['hhidpn', 'wave_num']

In [50]:
attributes = [
    'rabyear', # 出生年份
    'radage_y', # 死亡时年龄
    'radyear', # 死亡年份
    'radmonth', # 死亡月份
    'raracem', # 种族
    'rarelig', # 宗教信仰
    'rahispan', # 是否西班牙裔
    'ragender', # 性别
    'raedyrs', # 教育年限
    'rameduc', # 母亲教育年限
    'rafeduc', # 父亲教育年限
]

for att in attributes:
    df_cleaned[att] = HRS_Rand[att]

df_cleaned['status'] = 1 - df_cleaned['radyear'].isna().astype(int)

df_cleaned

,hhidpn,wave_num,rabyear,radage_y,radyear,radmonth,raracem,rarelig,rahispan,ragender,raedyrs,rameduc,rafeduc,status
0,1010,1,1938.0,57.0,1995.0,4.0,1.white/caucasian,4.none/no pref,0.not hispanic,1.male,16.0,10.0,10.0,1
1,2010,1,1934.0,67.0,2001.0,11.0,1.white/caucasian,1.protestant,0.not hispanic,2.female,8.0,8.5.8+ yrs,6.0,1
2,3010,7,1936.0,77.0,2013.0,8.0,1.white/caucasian,1.protestant,0.not hispanic,1.male,12.0,8.5.8+ yrs,12.0,1
3,3020,6,1938.0,77.0,2015.0,10.0,1.white/caucasian,1.protestant,0.not hispanic,2.female,16.0,12.0,8.5.8+ yrs,1
4,10001010,2,1939.0,NaN,NaN,NaN,1.white/caucasian,3.jewish,0.not hispanic,1.male,12.0,9.0,12.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42401,923525020,6,1955.0,NaN,NaN,NaN,2.black/african american,1.protestant,0.not hispanic,1.male,12.0,11.0,NaN,0
42402,952836010,6,1959.0,NaN,NaN,NaN,1.white/caucasian,2.catholic,1.hispanic,2.female,6.0,0.none,NaN,0
42403,958361010,4,1964.0,NaN,NaN,NaN,2.black/african american,1.protestant,0.not hispanic,1.male,11.0,NaN,NaN,0
42404,959733010,0,1961.0,NaN,NaN,NaN,2.black/african american,1.protestant,0.not hispanic,1.male,16.0,12.0,12.0,0


用cpi数据对df中的价格数据进行调整（以2020为基线）。

In [51]:
df_cpi = pd.read_csv('CPI.csv')
df_cpi = df_cpi[df_cpi.year % 2 == 0].reset_index(drop=True).drop('year', axis=1)
df_cpi['wave'] = range(1, 16)
df_cpi = df_cpi.sort_values('wave').reset_index(drop=True)
df_cpi

,cpi,cpi_2020,wave
0,140.300,0.542094,1
1,148.200,0.572619,2
2,156.900,0.606234,3
3,163.000,0.629803,4
4,172.200,0.665350,5
5,179.900,0.695102,6
6,188.900,0.729876,7
7,201.600,0.778947,8
8,215.303,0.831893,9
9,218.056,0.842530,10


In [52]:
for w in range(1, 15):
    df['rwgihr'][df.wave == w] = df['rwgihr'][df.wave == w] / float(df_cpi['cpi_2020'][df_cpi.wave == w])
    df['hatotb'][df.wave == w] = df['hatotb'][df.wave == w] / float(df_cpi['cpi_2020'][df_cpi.wave == w])

对时薪历史进行挖掘。

In [53]:
df_wg = df.groupby('hhidpn').agg({'rwgihr':['count', np.max, np.min]})
df_wg = pd.DataFrame(df_wg).reset_index()

In [54]:
lowwg = df.rwgihr[df.rlbrf == '1.works ft'].dropna().median() * 2 / 3
lowwg

14.290177924861815

In [55]:
df_wg.columns = ['_'.join(i) for i in df_wg.columns]
df_cleaned['wghist'] = 1 * (df_wg.rwgihr_amax < lowwg).astype(int) + \
    2 * (df_wg.rwgihr_amin >= lowwg).astype(int) + \
        3 * ((df_wg.rwgihr_amin < lowwg) & (df_wg.rwgihr_amax > lowwg)).astype(int)
df_cleaned['wghist'][df_cleaned['wghist'] == 1] = 'sustained'
df_cleaned['wghist'][df_cleaned['wghist'] == 2] = 'never'
df_cleaned['wghist'][df_cleaned['wghist'] == 3] = 'intermittent'
df_cleaned['wghist'][df_cleaned['wghist'] == 0] = np.nan

对雇佣情况进行挖掘。

In [56]:
df_emp = pd.DataFrame(df.groupby('hhidpn')['rinlbrf'].apply(lambda x: (len(x.dropna()) != 0) & np.all(x.dropna() == '1.yes'))).reset_index()
df_cleaned['employ'] = df_emp['rinlbrf'].astype(int)
df_cleaned['employ'][df_cleaned['employ'] == 1] = 'stable'
df_cleaned['employ'][df_cleaned['employ'] == 0] = 'fluctuated'

兼职的工作时间占比。

In [57]:
df_pt = pd.DataFrame(df.groupby('hhidpn')['rlbrf'].apply(
    lambda x: (x == '2.works pt').sum() / len(x.dropna()) if len(x.dropna()) != 0 else 0)).reset_index()
df_cleaned['prop_pt'] = df_pt['rlbrf'][:].copy(deep=True)

常住地挖掘。

In [58]:
df_live = pd.DataFrame(df.groupby('hhidpn')['rcenreg'].apply(
    lambda x: x.dropna().reset_index(drop=True)[0] if len(x.dropna().unique()) == 1 else '5.move' if len(x.dropna()) != 0 else np.nan)).reset_index()
df_cleaned['live'] = df_live['rcenreg'][:].copy(deep=True)
df_cleaned['live'][df_cleaned['live'] == '5.other'] = np.nan

婚姻状况挖掘。

In [59]:
df_m = pd.DataFrame(df.groupby('hhidpn')['rmstat'].apply(
    lambda x: x.dropna().reset_index(drop=True)[0] if len(x.dropna()) != 0 else np.nan)).reset_index()
df_cleaned['marriage'] = 'single'
df_cleaned['marriage'][df_m['rmstat'] == '1.married'] = 'married'
df_cleaned['marriage'][df_m['rmstat'] == '2.married,spouse absent'] = 'married'
df_cleaned['marriage'][df_m['rmstat'] == np.nan] = np.nan

总财富挖掘，用均值。

In [60]:
df_wealth = pd.DataFrame(df.groupby('hhidpn')['hatotb'].apply(
    lambda x: x.dropna()[x.dropna() > 0].mean() if len(x.dropna()[x.dropna() > 0]) != 0 else np.nan)).reset_index()
df_cleaned['wealth'] = df_wealth['hatotb'][:].copy(deep=True)

BMI，用均值。

In [61]:
df_bmi = pd.DataFrame(df.groupby('hhidpn')['rbmi'].apply(
    lambda x: x.dropna().mean() if len(x.dropna()) != 0 else np.nan)).reset_index()
df_cleaned['bmi'] = df_bmi['rbmi'][:].copy(deep=True)

健康状况自评。

In [62]:
df_sh = pd.DataFrame(df.groupby('hhidpn')['rshlt'].apply(
    lambda x: x.dropna().reset_index(drop=True)[0] if len(x.dropna()) != 0 else np.nan)).reset_index()
df_cleaned['self_health'] = df_sh['rshlt'][:].copy(deep=True)
df_cleaned['self_health'] = df_cleaned['self_health'].str.replace('1.excellent', 'good/very good/excellent')
df_cleaned['self_health'] = df_cleaned['self_health'].str.replace('2.very good', 'good/very good/excellent')
df_cleaned['self_health'] = df_cleaned['self_health'].str.replace('3.good', 'good/very good/excellent')
df_cleaned['self_health'] = df_cleaned['self_health'].str.replace('4.fair', 'fair/poor')
df_cleaned['self_health'] = df_cleaned['self_health'].str.replace('5.poor', 'fair/poor')

吸烟状况挖掘。

In [63]:
df_sm = pd.DataFrame(df.groupby('hhidpn')['rsmokev'].apply(
    lambda x: (len(x.dropna()) != 0) & np.any(x.dropna() == '1.yes'))).reset_index()
df_sm['never'] = pd.DataFrame(df.groupby('hhidpn')['rsmokev'].apply(
    lambda x: (len(x.dropna()) != 0) & np.all(x.dropna() == '0.no'))).reset_index()['rsmokev'].astype(int)
df_cleaned['smoke'] = df_sm['rsmokev'] + 2 * df_sm['never']
df_cleaned['smoke'][df_cleaned['smoke'] == 1] = 'ever'
df_cleaned['smoke'][df_cleaned['smoke'] == 2] = 'never'
df_cleaned['smoke'][df_cleaned['smoke'] == 0] = np.nan

是否有政府健康保险。

In [64]:
df_gov = pd.DataFrame(df.groupby('hhidpn')['rhigov'].apply(
    lambda x: (len(x.dropna()) != 0) & np.all(x.dropna() == '1.yes'))).reset_index()
df_gov['never'] = pd.DataFrame(df.groupby('hhidpn')['rhigov'].apply(
    lambda x: (len(x.dropna()) != 0) & np.all(x.dropna() == '0.no'))).reset_index()['rhigov'].astype(int)
df_gov['sometimes'] = pd.DataFrame(df.groupby('hhidpn')['rhigov'].apply(
    lambda x: (len(x.dropna()) != 0) & ~np.all(x.dropna() == '0.no') & ~np.all(x.dropna() == '1.yes'))).reset_index()['rhigov'].astype(int)
df_cleaned['insur_gov'] = df_gov['rhigov'] + 2 * df_gov['never'] + 3 * df_gov['sometimes']
df_cleaned['insur_gov'][df_cleaned['insur_gov'] == 1] = 'always'
df_cleaned['insur_gov'][df_cleaned['insur_gov'] == 2] = 'never'
df_cleaned['insur_gov'][df_cleaned['insur_gov'] == 3] = 'sometimes'
df_cleaned['insur_gov'][df_cleaned['insur_gov'] == 0] = np.nan

是否有公司的健康保险。

In [65]:
df_com = pd.DataFrame(df.groupby('hhidpn')['rcovr'].apply(
    lambda x: (len(x.dropna()) != 0) & np.all(x.dropna() == '1.yes'))).reset_index()
df_com['never'] = pd.DataFrame(df.groupby('hhidpn')['rcovr'].apply(
    lambda x: (len(x.dropna()) != 0) & np.all(x.dropna() == '0.no'))).reset_index()['rcovr'].astype(int)
df_com['sometimes'] = pd.DataFrame(df.groupby('hhidpn')['rcovr'].apply(
    lambda x: (len(x.dropna()) != 0) & ~np.all(x.dropna() == '0.no') & ~np.all(x.dropna() == '1.yes'))).reset_index()['rcovr'].astype(int)
df_cleaned['insur_com'] = df_com['rcovr'] + 2 * df_com['never'] + 3 * df_com['sometimes']
df_cleaned['insur_com'][df_cleaned['insur_com'] == 1] = 'always'
df_cleaned['insur_com'][df_cleaned['insur_com'] == 2] = 'never'
df_cleaned['insur_com'][df_cleaned['insur_com'] == 3] = 'sometimes'
df_cleaned['insur_com'][df_cleaned['insur_com'] == 0] = np.nan

增加年龄，2020年的年龄。

In [66]:
df_cleaned['age'] = 2020 - df_cleaned['rabyear']

增加发生事件的时间。

In [67]:
df_cleaned['event'] = 2022 - 1992
df_cleaned['event'][~(df_cleaned.radyear.isna() & df_cleaned.radmonth.isna())] = df_cleaned['radyear'] + df_cleaned['radmonth'] / 12 - 1992
df_cleaned['event'][(~df_cleaned.radyear.isna()) & df_cleaned.radmonth.isna()] = df_cleaned['radyear'] + 0.5 - 1992

删除没用的特征。

In [68]:
df_cleaned = df_cleaned.drop(['rabyear', 'radage_y', 'radyear', 'radmonth'], axis=1)
df_cleaned

,hhidpn,wave_num,raracem,rarelig,rahispan,ragender,raedyrs,rameduc,rafeduc,status,wghist,employ,prop_pt,live,marriage,wealth,bmi,self_health,smoke,insur_gov,insur_com,age,event
0,1010,1,1.white/caucasian,4.none/no pref,0.not hispanic,1.male,16.0,10.0,10.0,1,never,stable,0.000000,1.northeast,single,1.106818e+04,27.550000,fair/poor,ever,never,always,82.0,3.333333
1,2010,1,1.white/caucasian,1.protestant,0.not hispanic,2.female,8.0,8.5.8+ yrs,6.0,1,sustained,fluctuated,0.000000,2.midwest,single,9.208154e+01,16.780000,good/very good/excellent,ever,always,never,86.0,9.916667
2,3010,7,1.white/caucasian,1.protestant,0.not hispanic,1.male,12.0,8.5.8+ yrs,12.0,1,intermittent,fluctuated,0.000000,4.west,married,1.280150e+06,26.136364,fair/poor,never,sometimes,sometimes,84.0,21.666667
3,3020,6,1.white/caucasian,1.protestant,0.not hispanic,2.female,16.0,12.0,8.5.8+ yrs,1,sustained,fluctuated,0.000000,4.west,married,1.270952e+06,28.891667,good/very good/excellent,never,sometimes,never,82.0,23.833333
4,10001010,2,1.white/caucasian,3.jewish,0.not hispanic,1.male,12.0,9.0,12.0,0,sustained,fluctuated,0.000000,1.northeast,single,6.523271e+05,22.476923,good/very good/excellent,never,sometimes,never,81.0,30.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42401,923525020,6,2.black/african american,1.protestant,0.not hispanic,1.male,12.0,11.0,NaN,0,intermittent,stable,0.333333,3.south,married,8.707313e+04,21.833333,fair/poor,ever,sometimes,sometimes,65.0,30.000000
42402,952836010,6,1.white/caucasian,2.catholic,1.hispanic,2.female,6.0,0.none,NaN,0,intermittent,stable,0.000000,3.south,single,5.372509e+04,NaN,good/very good/excellent,never,never,sometimes,61.0,30.000000
42403,958361010,4,2.black/african american,1.protestant,0.not hispanic,1.male,11.0,NaN,NaN,0,never,stable,0.400000,1.northeast,single,1.017799e+05,25.840000,good/very good/excellent,ever,sometimes,sometimes,56.0,30.000000
42404,959733010,0,2.black/african american,1.protestant,0.not hispanic,1.male,16.0,12.0,12.0,0,NaN,fluctuated,0.000000,3.south,single,1.068211e+03,21.800000,fair/poor,ever,always,never,59.0,30.000000


将数据保存到本地。

In [69]:
df_cleaned.to_csv('df_cleaned.csv', index=False)

## 进一步清洗数据

In [70]:
df_final = pd.read_csv('df_cleaned.csv')

筛选时薪超过5个wave的数据。

In [71]:
df_final = df_final[df_final.wave_num > 5].reset_index(drop=True)
df_final

,hhidpn,wave_num,raracem,rarelig,rahispan,ragender,raedyrs,rameduc,rafeduc,status,wghist,employ,prop_pt,live,marriage,wealth,bmi,self_health,smoke,insur_gov,insur_com,age,event
0,3010,7,1.white/caucasian,1.protestant,0.not hispanic,1.male,12.0,8.5.8+ yrs,12.0,1,intermittent,fluctuated,0.000000,4.west,married,1.280150e+06,26.136364,fair/poor,never,sometimes,sometimes,84.0,21.666667
1,3020,6,1.white/caucasian,1.protestant,0.not hispanic,2.female,16.0,12.0,8.5.8+ yrs,1,sustained,fluctuated,0.000000,4.west,married,1.270952e+06,28.891667,good/very good/excellent,never,sometimes,never,82.0,23.833333
2,10013040,8,1.white/caucasian,3.jewish,0.not hispanic,2.female,13.0,12.0,11.0,0,never,fluctuated,0.200000,1.northeast,married,8.285465e+05,24.766667,good/very good/excellent,ever,sometimes,sometimes,73.0,30.000000
3,10038010,9,1.white/caucasian,3.jewish,0.not hispanic,1.male,16.0,12.0,12.0,0,never,fluctuated,0.066667,1.northeast,married,3.808126e+06,23.946667,good/very good/excellent,never,sometimes,sometimes,84.0,30.000000
4,10038040,12,1.white/caucasian,3.jewish,0.not hispanic,2.female,16.0,10.0,12.0,0,never,fluctuated,0.200000,1.northeast,married,3.808126e+06,22.420000,good/very good/excellent,ever,sometimes,sometimes,77.0,30.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6314,920753010,6,2.black/african american,1.protestant,0.not hispanic,2.female,12.0,10.0,NaN,0,sustained,stable,0.666667,3.south,married,7.853883e+02,26.716667,good/very good/excellent,never,never,sometimes,61.0,30.000000
6315,923497020,6,3.other,2.catholic,1.hispanic,2.female,12.0,6.0,6.0,0,never,stable,0.000000,4.west,married,1.874183e+05,35.166667,fair/poor,never,never,always,61.0,30.000000
6316,923498010,6,2.black/african american,1.protestant,0.not hispanic,2.female,10.0,12.0,NaN,0,never,stable,0.166667,4.west,single,1.032720e+04,29.400000,good/very good/excellent,ever,sometimes,never,65.0,30.000000
6317,923525020,6,2.black/african american,1.protestant,0.not hispanic,1.male,12.0,11.0,NaN,0,intermittent,stable,0.333333,3.south,married,8.707313e+04,21.833333,fair/poor,ever,sometimes,sometimes,65.0,30.000000


种族。

In [72]:
df_final['race'] = 1 * (df_final['rahispan'] == '1.hispanic').astype(int) + \
    2 * ((df_final['rahispan'] == '0.not hispanic') & (df_final['raracem'] == '1.white/caucasian')).astype(int) + \
        3 * ((df_final['rahispan'] == '0.not hispanic') & (df_final['raracem'] == '2.black/african american')).astype(int) + \
            4 * ((df_final['rahispan'] == '0.not hispanic') & (df_final['raracem'] == '3.other')).astype(int)
df_final['race'][df_final['race'] == 1] = 'hispan'
df_final['race'][df_final['race'] == 2] = 'nhwhite'
df_final['race'][df_final['race'] == 3] = 'nhblack'
df_final['race'][df_final['race'] == 4] = 'nhother'
df_final['race'][df_final['race'] == 0] = np.nan

性别。

In [73]:
df_final['gender'] = df_final['ragender']

宗教信仰。

In [74]:
df_final['religion'] = df_final['rarelig']

个人教育情况。

In [75]:
df_final['raedyrs'][df_final['raedyrs'] == '0.none'] = '0'
df_final['raedyrs'][df_final['raedyrs'] == '17.17+ yrs'] = '17.0'
df_final['raedyrs'] = df_final['raedyrs'].astype(float)
df_final['edu'] = (df_final['raedyrs'] <= 12).astype(int) + 2 * (df_final['raedyrs'] >= 13).astype(int)
df_final['edu'][df_final['edu'] == 1] = '<=12years'
df_final['edu'][df_final['edu'] == 2] = '>12years'
df_final['edu'][df_final['edu'] == 0] = np.nan

父母教育情况。

In [76]:
# 母亲教育情况
df_final['rameduc'][df_final['rameduc'] == '0.none'] = '0'
df_final['rameduc'][df_final['rameduc'] == '8.5.8+ yrs'] = '8.0'
df_final['rameduc'][df_final['rameduc'] == '17.17+ yrs'] = '17.0'
df_final['rameduc'] = df_final['rameduc'].astype(float)
# 父亲教育情况
df_final['rafeduc'][df_final['rafeduc'] == '0.none'] = '0'
df_final['rafeduc'][df_final['rafeduc'] == '8.5.8+ yrs'] = '8.0'
df_final['rafeduc'][df_final['rafeduc'] == '17.17+ yrs'] = '17.0'
df_final['rafeduc'] = df_final['rafeduc'].astype(float)
# 父母最高教育情况
df_final['ramf'] = df_final[['rafeduc', 'rameduc']].max(axis=1)
df_final['pedu'] = (df_final['ramf'] < 12).astype(int) + 2 * (df_final['ramf'] >= 12).astype(int)
df_final['pedu'][df_final['pedu'] == 1] = '<=12years'
df_final['pedu'][df_final['pedu'] == 2] = '>12years'
df_final['pedu'][df_final['pedu'] == 0] = np.nan

删除没用的变量。

In [77]:
df_final = df_final.drop(['rameduc', 'rafeduc', 'ramf', 'raedyrs', 'ragender', 'raracem', 'rahispan', 'rarelig'], axis=1)

In [78]:
df_final = df_final[['hhidpn', 'event', 'status', 'wghist', 'wave_num', 'prop_pt', 'employ',
                     'age', 'race', 'gender', 'edu', 'pedu', 'religion', 'live', 
                     'marriage', 'wealth', 'insur_gov', 'insur_com', 
                     'bmi', 'self_health', 'smoke']]

删除缺失值（317行）。

In [79]:
df_final = df_final.dropna().reset_index(drop=True)
df_final

,hhidpn,event,status,wghist,wave_num,prop_pt,employ,age,race,gender,edu,pedu,religion,live,marriage,wealth,insur_gov,insur_com,bmi,self_health,smoke
0,3010,21.666667,1,intermittent,7,0.000000,fluctuated,84.0,nhwhite,1.male,<=12years,>12years,1.protestant,4.west,married,1.280150e+06,sometimes,sometimes,26.136364,fair/poor,never
1,3020,23.833333,1,sustained,6,0.000000,fluctuated,82.0,nhwhite,2.female,>12years,>12years,1.protestant,4.west,married,1.270952e+06,sometimes,never,28.891667,good/very good/excellent,never
2,10013040,30.000000,0,never,8,0.200000,fluctuated,73.0,nhwhite,2.female,>12years,>12years,3.jewish,1.northeast,married,8.285465e+05,sometimes,sometimes,24.766667,good/very good/excellent,ever
3,10038010,30.000000,0,never,9,0.066667,fluctuated,84.0,nhwhite,1.male,>12years,>12years,3.jewish,1.northeast,married,3.808126e+06,sometimes,sometimes,23.946667,good/very good/excellent,never
4,10038040,30.000000,0,never,12,0.200000,fluctuated,77.0,nhwhite,2.female,>12years,>12years,3.jewish,1.northeast,married,3.808126e+06,sometimes,sometimes,22.420000,good/very good/excellent,ever
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,920736010,30.000000,0,intermittent,6,0.000000,stable,64.0,hispan,1.male,<=12years,<=12years,2.catholic,2.midwest,married,4.280459e+04,sometimes,never,26.050000,good/very good/excellent,ever
5998,920753010,30.000000,0,sustained,6,0.666667,stable,61.0,nhblack,2.female,<=12years,<=12years,1.protestant,3.south,married,7.853883e+02,never,sometimes,26.716667,good/very good/excellent,never
5999,923497020,30.000000,0,never,6,0.000000,stable,61.0,hispan,2.female,<=12years,<=12years,2.catholic,4.west,married,1.874183e+05,never,always,35.166667,fair/poor,never
6000,923498010,30.000000,0,never,6,0.166667,stable,65.0,nhblack,2.female,<=12years,>12years,1.protestant,4.west,single,1.032720e+04,sometimes,never,29.400000,good/very good/excellent,ever


将数据保存到本地。

In [80]:
df_final.to_csv('df_final.csv', index=False)